# Setup

In [ ]:
from chetan import SessionManager
from chetan.runtime import LocalRuntime, RemoteRuntime
from chetan.base import ChetanbaseClient

# rt = RemoteRuntime("https://runtime.snayu.ai") # Need runtime credentials CHETAN_REMOTE_SECRET
rt = LocalRuntime()

mgr = SessionManager(
    runtime=rt,
    base_client=ChetanbaseClient()
)

# Primitives
## Language Model

In [ ]:
mgr.lm["default"] = rt.LanguageModel(provider="azure-openai", model="gpt-4o-mini")

## Agent architecture
### Modules
- Server-side module
- Client-side module

In [ ]:

# ? Load from hub
modules = rt.Module.from_hub(
    "snayu/human-in-the-loop", # Automatically ask for human intervention
    "llamaindex/rag",
    
    "snayu/smriti-memory",
    # "zep/memory", # ? or
    # "letta/memory", # ? or
    
    "snayu/tool-recommender",
)

# ? Load from Git repo
modules.extend(
    rt.Module.from_git(
        "https://github.com/Snayu-AI/chetan-hierarchical-planner",
        # ...
    )
)

In [ ]:

# ? Load directly
# ! Only on LocalRuntime
# ! RemoteRuntime with severe latency warning

from somelibrary.modules import SomeModule

modules.extend(
    rt.Module.from_loaded(SomeModule())
)

### Agent Loop

In [ ]:
loop = rt.AgentLoop().use(*modules)

pfns = rt.process.functions
loop.process_sequence = pfns.generate(
    reasoning=False, max_tool_calls=-1
).execute_tool_calls()

mgr.agentloop["default"] = loop

## Tools
### MCP

In [ ]:
from mcp import StdioServerParameters

# ? Load MCP servers as tools
mgr.tools.load(
    rt.Tool.from_mcp(
        {
            "some_local_mcp": "/path/to/mcp", # relative to Runtime
            "some_remote_mcp": "https://remote-mcp.somewhere.io",
            "some_custom_stdio_mcp": StdioServerParameters(), # relative to Runtime
        }
    )
)

### Hub

In [ ]:

# ? Load from hub
mgr.tools.load(
    rt.Tool.from_hub(
        "tavily/tavily-search",
        "github/github-api",
        "microsoft/azure-portal-api",
        "google/youtube-videos-api",
    )
)

### Local

In [ ]:
from somewhere.tools import SomeTool

# ? Load directly
mgr.tools.load(
    rt.Tool.from_local(
        {
            "some_tool": SomeTool(),
            # ...
        }
    )
)

### Tool registry

In [ ]:

# * Instantiate all tools in root namespace
for tool in mgr.tools.items:
    mgr.tools.register("", tool)

# Entity
## Manifesto

In [ ]:
from chetan.manifesto import Manifesto
from chetan.manifesto.agentloop import AutoAgentLoop, tool_approval_condition


class SomeManifesto(Manifesto):
    agentloop = AutoAgentLoop(
        auto_tool_approval=tool_approval_condition.max_severity("medium").verbs(
            "read", "log"
        )
    )
    pass